In [1]:
import time
import tushare as ts
import datetime
import pandas as pd
import os
import tensorflow as tf
from collections import namedtuple
import numpy as np

In [2]:
if os.path.exists('601318.SH_5min.csv'):
    data = pd.read_csv('601318.SH_5min.csv', index_col=0)
    print('Read sussessful')
#     print('head: \n', data.head())
#     print('tail: \n', data.tail())
else:
    print('File not exist')

Read sussessful


In [3]:
data.head(5)

,ts_code,trade_time,open,close,high,low,vol,amount
0,601318.SH,2019-10-22 15:00:00,90.15,90.29,90.29,90.15,966545.0,87224082.0
1,601318.SH,2019-10-22 14:55:00,90.12,90.15,90.19,90.11,735530.0,66309263.0
2,601318.SH,2019-10-22 14:50:00,90.10,90.11,90.12,90.07,558033.0,50276229.0
3,601318.SH,2019-10-22 14:45:00,90.10,90.10,90.14,90.06,640738.0,57733317.0
4,601318.SH,2019-10-22 14:40:00,90.25,90.12,90.25,90.06,724000.0,65254508.0


In [4]:
def get_time(t):
    time = datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S')
    minus_9_30 = (int(time.strftime('%H'))-9) * 12 + int(time.strftime('%M')) / 5 - 6
    return minus_9_30 if minus_9_30 < 25 else minus_9_30 - 18

class Observations:
    def __init__(self, index, is_hold, wait_time, trade_price):
        # is_hold: 是否持有股票，1表示持有，0表示未持有
        # trade_price: 距离上次操作的时间（多少个5分钟）
        # trade_price: 上次交易价格
        
        self.index = index
        self.is_hold = is_hold 
        self.wait_time = wait_time
        self.trade_price = trade_price

    def values(self, history_data, length):
        # history_data: DataFram 索引越靠前日期越靠后
        # 返回数据为 length * 6 + 3， 前 length * 6 为每日的 time，open， close， high， low， vol-10000
        # 其中 time 为 0 到 48， 表示一天中的第几个5分钟
        # 最后三位分别是 is_hold * 100，即100为持仓, 持仓是否过夜，100为过夜
        
        recent_data = history_data[['trade_time', 'open', 'high', 'low', 'close', 'vol']][
            self.index: self.index+length]
        recent_data['vol'] = recent_data['vol']/10000
        recent_data['trade_time'] = recent_data['trade_time'].apply(lambda x: get_time(x))
        is_pass_night = self.wait_time > 48 or self.wait_time > recent_data['trade_time'].iloc[0]
        return np.hstack([np.array(recent_data.values).reshape(1,-1),
                          np.array([[self.is_hold*100, 100 if is_pass_night else 0, self.trade_price]])])
        
    def decode(self, history_data, length, log=False):
        recent_data = history_data[['trade_time', 'open', 'high', 'low', 'close', 'vol']][
            self.index: self.index+length]
        recent_data['trade_time'] = recent_data['trade_time'].apply(lambda x: get_time(x))

        if log:
            print('recent data is :\n', recent_data)
            print('')

            if self.is_hold:
                print('Hold stock for {} minutes， purchase price is {}.'.format(
                    self.wait_time*5, self.trade_price))
            else:
                print('Dosen\'t hold any thing.')
        return recent_data
        
    def __str__(self):
        return 'index: {}, is_hold: {}, wait_time: {}, trade_price: {}\n'.format(
            self.index, self.is_hold, self.wait_time, self.trade_price)
    
    def __repr__(self):
        return self.__str__()

In [5]:
# Observation test
# index， is_hold， wait_time， trade_price
# obs = Observations(*[1, 1, 0, 89])
# next_obs = Observations(*[0, 1, 1, 89])
# print(obs.values(data, 3))
# print(next_obs.values(data, 3))

In [6]:
class Actions:
    
    def __init__(self, action_prob):
        # 买、卖、持有的几率
        self.p_buy = action_prob[0]
        self.p_sell = action_prob[1] 
        self.p_hold = 1 - action_prob[0] - action_prob[1] 
        
        self.action_choose = np.random.choice(['buy', 'sell', 'hold'],
                                              p=[self.p_buy, self.p_sell, self.p_hold])
# Deubg 
        print('action_prob: {}, action is {}'.format(action_prob, self.action_choose))
    
    def choose(self):

        return self.action_choose

+ 注意： 交易默认只买 100 股，手续费默认

In [7]:
def calc_reward_batch(obs, next_obs, history_data):
    # obs 和 next_obs 为 Observation 类
    
    fee = obs.trade_price * 0.02 if next_obs.wait_time == 1 else 0
    if obs.is_hold == 1:
        delta_price = (next_obs.decode(history_data, 1).close.iloc[0]
                       - obs.decode(history_data, 1).close.iloc[0]) * 100
        return delta_price - fee
    else:
        return -fee
        

In [8]:
# 计算 reward 测试
# obs.decode(data, 3, log=True)
# next_obs.decode(data, 3, log=True)
# calc_reward_batch(obs, next_obs, data)

In [9]:
class Env:
    def __init__(self, hps, history_data):
        self._hps = hps
        self._history_data = history_data
        
        self._observations_dim = hps.days * 6 + 3
        self._actions_dim = 3
        return
    
    def reset(self):
        index = self._history_data.shape[0] - self._hps.days - 1
        return Observations(index=index, is_hold=0, wait_time=0, trade_price=0)
    
    def step(self, obs, action):
        # 输入为 Observations 类和 Actions 类
        # 返回值为 next observations， reward， done
        index, is_hold, wait_time, trade_price = obs.index, obs.is_hold, obs.wait_time, obs.trade_price
        done = True if index == 0 else False
        action_choose = action.choose()
        
        if is_hold == 1 and action_choose == 'sell':
            current_time = get_time(self._history_data['trade_time'].iloc[index])
            is_pass_night = wait_time>48 or wait_time>current_time
            if is_pass_night:
                is_hold = 0  # 卖掉了
                wait_time = 0  # 时间清0
                trade_price = self._history_data['close'].iloc[index]  # 以当前的收盘价为成交加个
            else:
                pass # 不做操作，类似 hold
        elif is_hold == 0 and action_choose == 'buy':
            is_hold = 1
            wait_time = 0
            trade_price = self._history_data['close'].iloc[index]
        else:
            pass # 不做操作
        
        next_obs = Observations(index-1, is_hold, wait_time+1, trade_price)

        return next_obs, calc_reward_batch(obs, next_obs, self._history_data), done
    
    @property
    def observations_dim(self):
        return self._observations_dim
    
    @property
    def actions_dim(self):
        return self._actions_dim

In [10]:
class DataSet:
    def __init__(self, hps, history_data):
        self._buffer = []
        self._length = 0
        self._hps = hps
        self._history_data = history_data
        return 
    
    def get_batch(self, nums):
        assert self._length > 1, 'Length of data is {} which is not enough. \
        Data need at least {}'.format(self._length, 2)
        
        rand_idx = np.random.randint(0, self._length-1, nums)
        obs = np.vstack([self._buffer[x].values(
            self._history_data, self._hps.days) for x in rand_idx])
        
        next_obs = np.vstack([self._buffer[x+1].values(
            self._history_data, self._hps.days) for x in rand_idx])
        
        reward = np.array([calc_reward_batch(self._buffer[x], self._buffer[x+1], self._history_data)
                           for x in rand_idx])
    
        return obs, next_obs, reward
    
    def add_data(self, obs):
        # obs 为 Observation 类
        self._buffer.append(obs)
        self._length += 1
        return
        

In [11]:
# DataSet test

# obs = Observations(*[1, 1, 0, 89])
# next_obs = Observations(*[0, 1, 1, 89])
# data_set = DataSet(hps, data)
# data_set.add_data(obs)
# data_set.add_data(obs)
# data_set.add_data(obs)
# print(data_set._buffer)
# obs.trade_price = 100
# print(data_set._buffer)
# print(obs)
# data_set.add_data(obs)
# obs, next_obs, reward = data_set.get_batch(2)
# print('obs: \n', obs)
# print('\nnext_obs: \n', next_obs)
# print('\nreward: \n', reward)

In [12]:
class Model:
    def __init__(self, env, hps):
        self._env = env
        self._hps = hps
        
        self.global_step = tf.Variable(0, name='global_step', trainable=False)
        self.action, self.Q, self.action_loss, self.Q_loss, self.action_train_opt, self.Q_train_opt = \
            self._build_graph()
        self._sess, self._summary_writer = self._sess_setup()
        return
    
    def train(self, iteration, data_set):
        for i in range(iteration):
            obs, next_obs, reward = data_set.get_batch(self._hps.batch_size)
            action_loss, Q_loss = self._train_one_step(obs, next_obs, reward)
            print('action_loss: {}, Q_loss : {}'.format(action_loss, Q_loss))

        return
    
    def test(self, data):
        return
    
    def predict(self, obs):
        action_prob = self._sess.run(self.action, {self._observations_ph: obs})
        return action_prob
    
    def _train_one_step(self, obs, next_obs, reward):
        feed_dict = {self._observations_ph: obs,
                    self._next_observations_ph: next_obs,
                    self._rewards_ph: reward}
        
        action_loss, Q_loss, _, _ = self._sess.run([self.action_loss, self.Q_loss,
                                                    self.action_train_opt, self.Q_train_opt],
                                                   feed_dict)
        return action_loss, Q_loss
    
    def _sess_setup(self):
        saver = tf.train.Saver(max_to_keep=3)
        sv = tf.train.Supervisor(logdir=self._hps.train_dir,
                   is_chief=True,
                   saver=saver,
                   summary_op=None,
                   save_summaries_secs=600, # save summaries for tensorboard every 60 secs
                   save_model_secs=600, # checkpoint every 600 secs
                   global_step=self.global_step,
                   init_feed_dict= None
                   )
        summary_writer = sv.summary_writer
        sess = sv.prepare_or_wait_for_session()
    
        return sess, summary_writer
    
    def _create_placeholders(self):
        observations_dim = self._env.observations_dim
        actions_dim = self._env.actions_dim

        
        self._observations_ph = tf.placeholder(
            tf.float32,
            shape=(None, observations_dim),
            name='observation',
        )
        self._next_observations_ph = tf.placeholder(
            tf.float32,
            shape=(None, observations_dim),
            name='next_observation',
        )
#         self._actions_ph = tf.placeholder(
#             tf.float32,
#             shape=(None, actions_dim),
#             name='actions',
#         )
        self._rewards_ph = tf.placeholder(
            tf.float32,
            shape=(None, ),
            name='rewards',
        )
        return
    
#     def _linear(self, arg, output_size, activation, scope=None, reuse=False):
#         input_size = arg.get_shape().as_list()[1]
#         print('input_size', input_size)
#         trunc_norm_init = tf.truncated_normal_initializer(stddev=self._hps.trunc_norm_init_std)
        
#         with tf.variable_scope(scope or "Linear", reuse=reuse):
#             matrix = tf.get_variable("Matrix", [input_size, output_size])
#             res = tf.matmul(arg, matrix)
#             bias_term = tf.get_variable("Bias", [output_size],
#                                         initializer=trunc_norm_init)
#         return activation(res + bias_term)
        
    def _action_Q_output(self, state, reuse=False):
        
        with tf.variable_scope('hidden_state', reuse=tf.AUTO_REUSE):
            hidden_states = tf.layers.dense(state, self._hps.hidden_dim,
                                           activation=tf.nn.sigmoid, name='state_hidden_layer')
            
        with tf.variable_scope('action_output', reuse=tf.AUTO_REUSE):
            actions = tf.nn.softmax(tf.layers.dense(hidden_states, self._env.actions_dim,
                                                    activation=tf.nn.sigmoid, name='action_output_layer'))
            
        with tf.variable_scope('Q_output', reuse=tf.AUTO_REUSE):
            Q = tf.layers.dense(tf.concat([hidden_states, actions], axis=1), 1,
                                           activation=None, name='Q_output_layer')
        return hidden_states, actions, Q
        
    def _build_graph(self):
        self._create_placeholders()
        _, action, Q = self._action_Q_output(self._observations_ph)
        _, _, next_Q = self._action_Q_output(self._next_observations_ph)
        
        # Calculate action loss and Q loss
        action_loss = -tf.reduce_sum(tf.squeeze(Q), axis=0)

        Q_loss = tf.reduce_sum(self._rewards_ph + tf.squeeze(self._hps.gamma * next_Q - Q), axis=0)

        
        # Get update option
        t_vars = tf.trainable_variables()
        action_vars = [var for var in t_vars 
                       if var.name.startswith('hidden_state') or var.name.startswith('action_output')]
        
        Q_vars = [var for var in t_vars 
                  if var.name.startswith('hidden_state') or var.name.startswith('Q_output')] 
        
        action_train_opt = tf.train.AdamOptimizer(self._hps.learning_rate).minimize(
            action_loss, var_list = action_vars)
        
        Q_train_opt = tf.train.AdamOptimizer(self._hps.learning_rate).minimize(
            action_loss, var_list = Q_vars)
        
        return action, Q, action_loss, Q_loss, action_train_opt, Q_train_opt

        

In [13]:
class Agent:
    def __init__(self, hps, env, history_data):
        self._hps = hps
        self._env = env
        self._history_data = history_data
        self._data_set = DataSet(hps, history_data)
        self._model = Model(env, hps)
        return
    
    def step(self, obs):
        self._data_set.add_data(obs)
        action_prob = self._model.predict(obs.values(self._history_data, self._hps.days))
        action = Actions(action_prob[0])
        
        if self._data_set._length > 20:
            self._model.train(1, self._data_set)
            
        return action
    

In [14]:
hps = {'trunc_norm_init_std': 1e-4,
      'hidden_dim': 20,
      'train_dir': './model',
      'gamma': 0.99,
      'learning_rate': 0.003,
      'batch_size': 10,
      'days': 20}
hps = namedtuple("HParams", hps.keys())(**hps)

In [15]:
# For model test
# env = Env(hps, data)
# obs = env.reset()
# obs.values(data, hps.days)

In [16]:
env = Env(hps, data)
agent = Agent(hps, env, data)

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Restoring parameters from ./model/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path ./model/model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:global_step/sec: 0


In [1]:
obs = env.reset()
rewards = []
reward_sum = 0
n = 100000
for i in range(n):
    print('{}/{}'.format(i, n))
    action = agent.step(obs)
    obs, reward, done = env.step(obs, action)
    rewards.append(reward)
    reward_sum += reward
    print('reward is {:.2f}, reward sum is {:.2f}, time is {}, close is {}\n'.format(
        reward, reward_sum, obs.decode(data, 1)['trade_time'].iloc[0], obs.decode(data, 1)['close'].iloc[0]))


In [ ]:
agent._data_set._buffer